In [1]:
# Import packages
import logging
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import os
import sys
import re
import string
import concurrent
from typing import List, Optional
from ignore_SSL_errors import no_ssl_verification
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
# Initiallze logging
logging.basicConfig(format='%(asctime)s: %(message)s',
                    datefmt='%Y-%m-%d %I:%M:%S %p',
                    handlers=[logging.StreamHandler()],
                    level=logging.INFO)
logging.info('Format check')


2022-02-07 11:22:32 AM: Format check


In [3]:
# Import list of euronext stocks => https://live.euronext.com/en/products/equities/list
# Important to remove rows 1 to 4 from csv file as they will create an error
file_name = "Euronext_Equities_2022-01-11.csv"
fields_list = ['Name', 'ISIN', 'Symbol', 'Market', 'Trading Currency']
stock_info_euronext = pd.read_csv(file_name, sep=';', usecols=fields_list)
renaming_dict = {
    'Name': 'stock_name',
    'ISIN': 'stock_isin',
    'Symbol': 'stock_euronext_symbol',
    'Market': 'stock_exchange_markets',
    'Trading Currency': 'stock_trading_currency'
}
stock_info_euronext.rename(columns=renaming_dict, inplace=True)
stock_info_euronext.head()


,stock_name,stock_isin,stock_euronext_symbol,stock_exchange_markets,stock_trading_currency
0,1000MERCIS,FR0010285965,ALMIL,Euronext Growth Paris,EUR
1,2020 BULKERS,BMG9156K1018,2020,Oslo B�rs,NOK
2,2CRSI,FR0013341781,2CRSI,Euronext Paris,EUR
3,2MX ORGANIC,FR0014000T90,2MX,Euronext Paris,EUR
4,2MX ORGANIC BS,FR0014000TB2,2MXBS,Euronext Paris,EUR


In [4]:
# Correct special character is Oslo Bors if needed, might not be needed if you encoding is correct on the csv
stock_info_euronext['stock_exchange_markets'] = stock_info_euronext['stock_exchange_markets'].str.replace(
    '�', 'ø')


In [5]:
# Get distinct stock details, we will need to try map this accordingly
stock_info_euronext['stock_exchange_markets'].unique()


array(['Euronext Growth Paris', 'Oslo Børs', 'Euronext Paris',
       'Euronext Expand Oslo', 'Euronext Access Paris',
       'Euronext Amsterdam', 'Euronext Growth Oslo', 'Euronext Brussels',
       'Euronext Expert Market', 'Euronext Brussels, Paris',
       'Euronext Paris, Amsterdam', 'Euronext Paris, Brussels',
       'Euronext Brussels, Amsterdam', 'Euronext Access Lisbon',
       'Euronext Amsterdam, Brussels', 'Euronext Dublin',
       'Euronext Lisbon', 'Euronext Amsterdam, Brussels, Paris',
       'Euronext Amsterdam, Paris', 'Traded not listed Brussels',
       'Euronext Growth Paris, Brussels', 'Euronext Growth Brussels',
       'Euronext Access Brussels', 'Euronext Growth Dublin',
       'Euronext Paris, Amsterdam, Brussels', 'Euronext Growth Lisbon'],
      dtype=object)

In [6]:
# Create a mapping for the exchange cities to yahoo codes
stock_exchange_markets_to_main_city_mapping = {
    'Amsterdam':'AS',
    'Brussels':'BR',
    'Dublin':'IR',
    'Lisbon':'LS',
    'Oslo':'OL',
    'Paris':'PA' 
    }

In [7]:
# Create a unique list of cities
cities_list = list(stock_exchange_markets_to_main_city_mapping.keys())
cities_list

['Amsterdam', 'Brussels', 'Dublin', 'Lisbon', 'Oslo', 'Paris']

In [8]:
# Function that takes in a string (input_text) and a list of different cities
# Returns the first city mentioned in the string that is in the cities list
def return_first_city(input_text: str, cities_list: List[str]) -> str:
    split_word_list = re.sub('['+string.punctuation+']', '', input_text).split()
    for word in split_word_list:
      if word in cities_list:
        result = word
        break
      else:
        result = "No city mentioned"
    return result


# Quick test to see required logic works
assert return_first_city('Amsterdam, Paris, Brussels', cities_list) == 'Amsterdam', 'Check logic'
assert return_first_city('Brussels, Amsterdam, Paris, Brussels',
                         cities_list) == 'Brussels', 'Check logic'


In [9]:
# Create column to get the main city
stock_info_euronext['main_city'] = stock_info_euronext['stock_exchange_markets'].apply(
    lambda x: return_first_city(x, cities_list))
stock_info_euronext.head()


,stock_name,stock_isin,stock_euronext_symbol,stock_exchange_markets,stock_trading_currency,main_city
0,1000MERCIS,FR0010285965,ALMIL,Euronext Growth Paris,EUR,Paris
1,2020 BULKERS,BMG9156K1018,2020,Oslo Børs,NOK,Oslo
2,2CRSI,FR0013341781,2CRSI,Euronext Paris,EUR,Paris
3,2MX ORGANIC,FR0014000T90,2MX,Euronext Paris,EUR,Paris
4,2MX ORGANIC BS,FR0014000TB2,2MXBS,Euronext Paris,EUR,Paris


In [10]:
# Get the Yahoo city mapping
stock_info_euronext['yahoo_city_code'] = stock_info_euronext['main_city'].map(
    stock_exchange_markets_to_main_city_mapping)
stock_info_euronext.head()


,stock_name,stock_isin,stock_euronext_symbol,stock_exchange_markets,stock_trading_currency,main_city,yahoo_city_code
0,1000MERCIS,FR0010285965,ALMIL,Euronext Growth Paris,EUR,Paris,PA
1,2020 BULKERS,BMG9156K1018,2020,Oslo Børs,NOK,Oslo,OL
2,2CRSI,FR0013341781,2CRSI,Euronext Paris,EUR,Paris,PA
3,2MX ORGANIC,FR0014000T90,2MX,Euronext Paris,EUR,Paris,PA
4,2MX ORGANIC BS,FR0014000TB2,2MXBS,Euronext Paris,EUR,Paris,PA


In [11]:
# Create the full city code
stock_info_euronext['yahoo_code'] = stock_info_euronext['stock_euronext_symbol'].str.cat(
    stock_info_euronext['yahoo_city_code'], sep='.')
stock_info_euronext.head()


,stock_name,stock_isin,stock_euronext_symbol,stock_exchange_markets,stock_trading_currency,main_city,yahoo_city_code,yahoo_code
0,1000MERCIS,FR0010285965,ALMIL,Euronext Growth Paris,EUR,Paris,PA,ALMIL.PA
1,2020 BULKERS,BMG9156K1018,2020,Oslo Børs,NOK,Oslo,OL,2020.OL
2,2CRSI,FR0013341781,2CRSI,Euronext Paris,EUR,Paris,PA,2CRSI.PA
3,2MX ORGANIC,FR0014000T90,2MX,Euronext Paris,EUR,Paris,PA,2MX.PA
4,2MX ORGANIC BS,FR0014000TB2,2MXBS,Euronext Paris,EUR,Paris,PA,2MXBS.PA


In [12]:
# Check all yahoo codes are unique
stock_info_euronext['yahoo_code'].describe()


count         1690
unique        1690
top       ALMIL.PA
freq             1
Name: yahoo_code, dtype: object

In [13]:
# Check all ISN codes are unique
stock_info_euronext['stock_isin'].describe()


count             1790
unique            1787
top       NL0006294274
freq                 2
Name: stock_isin, dtype: object

In [14]:
# Show duplicates
stock_info_euronext[stock_info_euronext.duplicated(['stock_isin'], keep=False)]


,stock_name,stock_isin,stock_euronext_symbol,stock_exchange_markets,stock_trading_currency,main_city,yahoo_city_code,yahoo_code
482,DIAGEO,GB0002374006,DGE,Euronext Paris,EUR,Paris,PA,DGE.PA
483,DIAGEO PLC,GB0002374006,GUI1,Euronext Dublin,EUR,Dublin,IR,GUI1.IR
578,ENVIPCO,NL0015000GX8,ENVI,Euronext Amsterdam,EUR,Amsterdam,AS,ENVI.AS
579,ENVIPCO HOLDING,NL0015000GX8,ENVIP,Euronext Growth Oslo,NOK,Oslo,OL,ENVIP.OL
612,EURONEXT,NL0006294274,ENX,"Euronext Paris, Amsterdam, Brussels",EUR,Paris,PA,ENX.PA
613,EURONEXT,NL0006294274,ENXP,Euronext Lisbon,EUR,Lisbon,LS,ENXP.LS


In [15]:
# Keep first occurence and remove other duplicates
stock_info_euronext = stock_info_euronext.drop_duplicates(['stock_isin'], keep='first')
stock_info_euronext[stock_info_euronext.duplicated(['stock_isin'], keep=False)]


,stock_name,stock_isin,stock_euronext_symbol,stock_exchange_markets,stock_trading_currency,main_city,yahoo_city_code,yahoo_code


In [16]:
stock_info_euronext['stock_isin'].describe()


count             1787
unique            1787
top       FR0010285965
freq                 1
Name: stock_isin, dtype: object

In [17]:
# Create a function that extracts price data from yahoo using a ticker code
def get_yahoo_data(ticker_and_isin: List[str]) -> Optional[pd.DataFrame]:
    yahoo_code = ticker_and_isin[0]
    stock_isin = ticker_and_isin[1]
    try:
        ticker = yf.Ticker(yahoo_code)
    except:
        ticker = None
    # Get some basic stock info
    try:
        stock_info = ticker.info
    except:
        stock_info = None
    try: 
        yahoo_sector = stock_info['sector']
    except:
        yahoo_sector = None
    try:
        yahoo_industry = stock_info['industry']
    except:
        yahoo_industry = None
    try:
        yahoo_full_time_employees = stock_info['fullTimeEmployees']
    except:
        yahoo_full_time_employees = None
    try:
        yahoo_shares_outstanding = stock_info['sharesOutstanding']
    except:
        yahoo_shares_outstanding = None
    if yahoo_sector or yahoo_industry or yahoo_full_time_employees or yahoo_shares_outstanding:
        stock_info_dict = {
            'stock_isin' : stock_isin,
            'yahoo_sector': [yahoo_sector],
            'yahoo_industry': [yahoo_industry],
            'yahoo_full_time_employees': [yahoo_full_time_employees],
            'yahoo_shares_outstanding': [yahoo_shares_outstanding]
        }
        stock_info = pd.DataFrame.from_dict(stock_info_dict)
    else:
        stock_info = pd.DataFrame()
    try:
        # Get the return history
        prices = ticker.history(period="5y")
        renaming_dict = {
            'Date': 'date',
            'Open': 'open_price',
            'High': 'high_price',
            'Low': 'low_price',
            'Close': 'close_price',
            'Adj Close': 'adjusted_close_price',
            'Volume': 'trading_volume',
            'Dividends': 'dividents',
            'Stock Splits': 'stock_splits',
        }
        prices.reset_index(inplace=True)
        prices.rename(columns=renaming_dict, inplace=True)
        prices['stock_isin'] = stock_isin
    except:
        prices = pd.DataFrame()
    # Get financial statement history
    try:
        financials = ticker.financials
        financials.index.set_names(['financial_statement_measure'], inplace=True)
        financials.reset_index(inplace=True)
        financials = pd.melt(financials, id_vars='financial_statement_measure',
                                    var_name='date', value_name='amount')
        financials['stock_isin'] = stock_isin
    except:
        financials = pd.DataFrame()
    # Get earnings history
    try:
        earnings = ticker.earnings
        earnings.index.set_names(['year'], inplace=True)
        earnings.reset_index(inplace=True)
        earnings = pd.melt(earnings, id_vars='year',
                                        var_name='earnings_measure', value_name='amount')
        earnings['stock_isin'] = stock_isin
    except:
        earnings = pd.DataFrame()
    # Get balance sheet history
    try:
        balance_sheet = ticker.balance_sheet
        balance_sheet.index.set_names(['balance_sheet_measure'], inplace=True)
        balance_sheet.reset_index(inplace=True)
        balance_sheet = pd.melt(balance_sheet, id_vars='balance_sheet_measure',
                                        var_name='date', value_name='amount')
        balance_sheet['stock_isin'] = stock_isin
    except:
        balance_sheet = pd.DataFrame()
    # Get cashflow history
    try:
        cashflow = ticker.cashflow
        cashflow.index.set_names(['cash_flow_measure'], inplace=True)
        cashflow.reset_index(inplace=True)
        cashflow = pd.melt(cashflow, id_vars='cash_flow_measure',
                                        var_name='date', value_name='amount')
        cashflow['stock_isin'] = stock_isin
    except:
        cashflow = pd.DataFrame()
    return stock_info, prices, financials, earnings, balance_sheet, cashflow




In [18]:
# Test by extracting Apple stock returns
with no_ssl_verification():
    stock_info, prices, financials, earnings, balance_sheet, cashflow = get_yahoo_data(
        ['AAPL', 'US0378331005'])
print('Stock info:')
print(stock_info.head())
print('----------------------------------------------------------')
print('Historical prices:')
print(prices.head())
print('----------------------------------------------------------')
print('Historical financials:')
print(financials.head())
print('----------------------------------------------------------')
print('Historical earnings:')
print(earnings.head())
print('----------------------------------------------------------')
print('Historical balance sheet:')
print(balance_sheet.head())
print('----------------------------------------------------------')
print('Historical cashflow:')
print(cashflow.head())
print('----------------------------------------------------------')


Stock info:
     stock_isin yahoo_sector        yahoo_industry  yahoo_full_time_employees  yahoo_shares_outstanding
0  US0378331005   Technology  Consumer Electronics                     100000               16319399936
----------------------------------------------------------
Historical prices:
        date  open_price  high_price  low_price  close_price  trading_volume  dividents  stock_splits    stock_isin
0 2017-02-06      30.348      30.670     30.294       30.621       107383600      0.000         0.000  US0378331005
1 2017-02-07      30.680      31.044     30.658       30.912       152735200      0.000         0.000  US0378331005
2 2017-02-08      30.870      31.074     30.839       31.032        92016400      0.000         0.000  US0378331005
3 2017-02-09      31.075      31.264     30.950       31.256       113399600      0.142         0.000  US0378331005
4 2017-02-10      31.266      31.379     31.169       31.186        80262000      0.000         0.000  US0378331005
------

In [19]:
# Prepare list of ticker information to pass into our iterator function.
relevant_tickers = stock_info_euronext[[
    'yahoo_code', 'stock_isin']].dropna()
tickers = relevant_tickers.to_numpy().tolist()
len(tickers)

1687

In [20]:
# Define function to run through all the ticker codes and extract price data
# https://docs.python.org/3/library/concurrent.futures.html
def iterate_tickers(tickers: List[List[str]])-> List[pd.DataFrame]:
    stock_info_list, prices_list, financials_list, earnings_list, balance_sheet_list, cashflow_list = (list() for i in range(6))
    no_of_tickers = len(tickers)
    logging.info(f'Starting extraction of data for {no_of_tickers} tickers')
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        # Start the load operations and mark each future with its ticker and isn number
        download_ticker_info = {executor.submit(
            get_yahoo_data, ticker_and_isn): ticker_and_isn for ticker_and_isn in tickers}
        for future in concurrent.futures.as_completed(download_ticker_info):
            current_download = download_ticker_info[future]
            stock_info, prices, financials, earnings, balance_sheet, cashflow = future.result()
            stock_info_list.append(stock_info)
            prices_list.append(prices)
            financials_list.append(financials)
            earnings_list.append(earnings)
            balance_sheet_list.append(balance_sheet)
            cashflow_list.append(cashflow)
            # Check mark \u2713 cross mark \u2717
            # Retrieval status
            SI = '\u2717' if stock_info.empty else '\u2713'
            F = '\u2717' if financials.empty else '\u2713'
            E = '\u2717' if earnings.empty else '\u2713'
            BS = '\u2717' if balance_sheet.empty else '\u2713'
            CF = '\u2717' if cashflow.empty else '\u2713'
            logging.info(f'{current_download}: SI{SI},F{F},E{E},BS{BS},CF{CF}')
    return stock_info_list, prices_list, financials_list, earnings_list, balance_sheet_list, cashflow_list


In [21]:
# Loop through all tickers to acquire data
with no_ssl_verification():
  stock_info_list, price_list, financials_list, earnings_list, balance_sheet_list, cashflow_list = iterate_tickers(
      tickers)

2022-02-07 11:22:46 AM: Starting extraction of data for 1687 tickers
2022-02-07 11:23:01 AM: ['2MXBS.PA', 'FR0014000TB2']: SI✗,F✗,E✗,BS✗,CF✗
2022-02-07 11:23:01 AM: ['ALMIL.PA', 'FR0010285965']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:23:02 AM: ['2020.OL', 'BMG9156K1018']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:23:02 AM: ['2MX.PA', 'FR0014000T90']: SI✓,F✗,E✗,BS✗,CF✗
2022-02-07 11:23:03 AM: ['2CRSI.PA', 'FR0013341781']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:23:21 AM: ['AASB.OL', 'NO0010672181']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-07 11:23:23 AM: ['MLATV.PA', 'FR0010050773']: SI✗,F✗,E✗,BS✗,CF✗
2022-02-07 11:23:24 AM: ['AALB.AS', 'NL0000852564']: SI✓,F✓,E✓,BS✓,CF✓


- MLATV.PA: No data found, symbol may be delisted


2022-02-07 11:23:26 AM: ['ASP.PA', 'FR0000076887']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:23:30 AM: ['5PG.OL', 'DK0060945467']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:23:42 AM: ['ABI.BR', 'BE0974293251']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:23:43 AM: ['AB.PA', 'FR0010557264']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:23:48 AM: ['ABEO.PA', 'FR0013185857']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:23:49 AM: ['ABCA.PA', 'FR0004040608']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:23:52 AM: ['ABG.OL', 'NO0003021909']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:24:04 AM: ['ABNX.PA', 'FR0012616852']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:24:04 AM: ['ABVX.PA', 'FR0012333284']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:24:10 AM: ['ABN.AS', 'NL0011540547']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:24:10 AM: ['ABO.BR', 'BE0974278104']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:24:21 AM: ['ACPH.BR', 'GB00BYWF9Y76']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:24:23 AM: ['ACAN.PA', 'FR0000064602']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-07 11:24:28 AM: ['ACCEL.AS', 'NL0009767532']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11

- CUR.LS: No data found, symbol may be delisted


2022-02-07 11:27:27 AM: ['A5G.IR', 'IE00BF0L3536']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:27:33 AM: ['AF.PA', 'FR0000031122']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:27:36 AM: ['AI.PA', 'FR0000120073']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:27:44 AM: ['MLAIM.PA', 'FR0013285103']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-07 11:27:45 AM: ['AIR.PA', 'NL0000235190']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:27:47 AM: ['AIRX.OL', 'NO0010895568']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:27:52 AM: ['ALAIR.PA', 'IT0005450819']: SI✓,F✗,E✗,BS✗,CF✗
2022-02-07 11:27:55 AM: ['MLAIR.PA', 'FR0014003V77']: SI✓,F✗,E✗,BS✗,CF✗
2022-02-07 11:28:05 AM: ['AKAST.OL', 'NO0010215684']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:28:08 AM: ['AKER.OL', 'NO0010234552']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:28:11 AM: ['AKBM.OL', 'NO0010886625']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:28:14 AM: ['AJAX.AS', 'NL0000018034']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:28:16 AM: ['AKRBP.OL', 'NO0010345853']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 11:28:25 AM: ['ACC.OL', 'NO0010890304']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-

...

2022-02-07 02:10:55 PM: ['VIV.PA', 'FR0000127771']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:10:57 PM: ['VVY.AS', 'NL00150002Q7']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:11:04 PM: ['ALVGO.PA', 'FR0011532225']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:11:06 PM: ['VWA.BR', 'DE0007664005']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-07 02:11:10 PM: ['VWAP.BR', 'DE0007664039']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-07 02:11:18 PM: ['VTA.AS', 'GG00B1GHHH78']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:11:22 PM: ['VOLUE.OL', 'NO0010894603']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:11:23 PM: ['VLTSA.PA', 'FR0011995588']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:11:28 PM: ['VPK.AS', 'NL0009432491']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:11:31 PM: ['VVL.OL', 'NO0003025009']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:11:42 PM: ['VOW.OL', 'NO0010708068']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:11:42 PM: ['VGM.OL', 'NO0011037483']: SI✓,F✗,E✗,BS✗,CF✗
2022-02-07 02:11:43 PM: ['ALVDM.PA', 'FR0004045847']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:11:49 PM: ['VRAP.PA', 'FR0000062796']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07

- MLWEL.PA: No data found for this date range, symbol may be delisted


2022-02-07 02:12:49 PM: ['WEHB.BR', 'BE0003724383']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:12:58 PM: ['WEST.OL', 'NO0010768096']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-07 02:13:05 PM: ['MLWEY.PA', 'FR0010768770']: SI✓,F✗,E✗,BS✗,CF✗
2022-02-07 02:13:06 PM: ['ROCK.BR', 'BE0974401334']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:13:07 PM: ['MLWRC.PA', 'ES0105399002']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-07 02:13:09 PM: ['WWI.OL', 'NO0010571698']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:13:18 PM: ['WWIB.OL', 'NO0010576010']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:13:25 PM: ['ALWF.PA', 'FR0014000P11']: SI✓,F✗,E✗,BS✗,CF✗
2022-02-07 02:13:26 PM: ['WILS.OL', 'NO0010252356']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:13:28 PM: ['ALWIT.PA', 'FR0013143872']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:13:32 PM: ['MLWIZ.PA', 'FR00140047H7']: SI✓,F✗,E✗,BS✗,CF✗
2022-02-07 02:13:41 PM: ['WOLE.BR', 'BE0003600112']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-07 02:13:45 PM: ['WOLS.BR', 'BE0003571792']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-07 02:13:51 PM: ['WLN.PA', 'FR0011981968']: SI✓,F✓,E✓,BS✓,CF✓
2022

- CV.PA: No data found, symbol may be delisted


2022-02-07 02:14:50 PM: ['ZENA.OL', 'CA98936C1068']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-07 02:14:51 PM: ['ZWIPE.OL', 'NO0010721277']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:14:53 PM: ['ORN.OL', 'NO0010934748']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-07 02:14:54 PM: ['ZEN.BR', 'BE0003809267']: SI✓,F✓,E✗,BS✓,CF✓


In [22]:
# Concat lists with dataframes if need be and save all files to csv
# Euronext stock info
stock_info_euronext.to_csv(
    'raw_data/stock_info_euronext.csv', sep='|', index=False)
# Yahoo stock info
stock_info_yahoo = pd.concat(stock_info_list)
stock_info_yahoo.to_csv(
    'raw_data/stock_info_yahoo.csv', sep='|', index=False)
# Price history
price_data = pd.concat(price_list)
price_data.to_csv(
    'raw_data/price_yahoo.csv', sep='|', index=False)
# Financials history
financials_data = pd.concat(financials_list)
financials_data.to_csv(
    'raw_data/financials_yahoo.csv', sep='|', index=False)
# Earnings history
earnings_data = pd.concat(earnings_list)
earnings_data.to_csv(
    'raw_data/earnings_yahoo.csv', sep='|', index=False)
# Balance sheet history
balance_sheet__data = pd.concat(balance_sheet_list)
balance_sheet__data.to_csv(
    'raw_data/balance_sheet_yahoo.csv', sep='|', index=False)
# Cash flow history
cashflow_data = pd.concat(cashflow_list)
cashflow_data.to_csv(
    'raw_data/cashflow_yahoo.csv', sep='|', index=False)

